In [ ]:
import numpy as np
import os
import plotly.graph_objects as go

from chimera_fgo.util.kitti import process_kitti_gt, load_icp_results
from chimera_fgo.util.plot import plot_trajectories, plot_MC

%load_ext autoreload
%autoreload 2

In [ ]:
kitti_seq = '0034'
MAX_BIAS = 1
start_idx = 1550 if kitti_seq == '0028' else 0

In [ ]:
# Load ground truth
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

### Plot Lidar odometry, blind FGO, and FGO

In [ ]:
# Lidar odometry
data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

In [ ]:
# Load results
run_name = 'fgo_100m_5runs_2023-01-19-0507'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_results = np.load(os.path.join(results_path, results_files[0]))
fgo_positions = fgo_results['positions']

run_name = 'fgo_100m_5runs_blind_2023-01-19-0528'
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, run_name)
results_files = os.listdir(results_path)
fgo_blind_results = np.load(os.path.join(results_path, results_files[0]))
fgo_blind_positions = fgo_blind_results['positions']

spoofed_positions = fgo_results['spoofed']

In [ ]:
# Plot
ATTACK_START = 1000
N = 1900
fgo_traj = go.Scatter(x=fgo_positions[:,0], y=fgo_positions[:,1], name='FGO', line=dict(color='blue'))
fgo_blind_traj = go.Scatter(x=fgo_blind_positions[:,0], y=fgo_blind_positions[:,1], name='FGO blind', line=dict(color='orange'))
gt_traj = go.Scatter(x=gt_enu[:N,0], y=gt_enu[:N,1], name='Ground-truth', line=dict(color='black'))
lidar_traj = go.Scatter(x=lidar_positions[:N,0], y=lidar_positions[:N,1], name='Lidar odometry', line=dict(color='green'))
start = go.Scatter(x=[0], y=[0], name='Start', mode='markers', marker=dict(size=10, color='blue'), showlegend=False)
plot_data = [fgo_traj, fgo_blind_traj, lidar_traj, gt_traj, start]

spoof_traj = go.Scatter(x=spoofed_positions[:,0], y=spoofed_positions[:,1], 
    name='Spoofed', line=dict(color='red', dash='dash')) 
spoof_start = go.Scatter(x=[spoofed_positions[ATTACK_START,0]], y=[spoofed_positions[ATTACK_START,1]], 
    name='Spoofing start', mode='markers', marker=dict(size=10, color='red'), showlegend=False)
plot_data += [spoof_traj, spoof_start]


# detect = go.Scatter(x=[graph_positions[detect_idx,0]], y=[graph_positions[detect_idx,1]], 
#     name='Detection', mode='markers', hovertext=str(detect_idx), marker=dict(size=10, color='green'), showlegend=True)
# plot_data += [detect]

fig = go.Figure(data=plot_data)
fig.update_layout(width=1000, height=1000, xaxis_title='East [m]', yaxis_title='North [m]')
# Move legend into plot
fig.update_layout(legend=dict(x=0.02, y=0.98), font=dict(size=18))
fig.update_yaxes(
    scaleanchor = "x",
    scaleratio = 1,
  )
fig.update_xaxes(autorange=True)
fig.show()

### Monte carlo trajectory plots

In [ ]:
kitti_seq = '0027'
run_name = 'fgo_200m_5runs_300w'

In [ ]:
#results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, 'old', run_name)
results_path = os.path.join(os.getcwd(), '..', 'results', kitti_seq, '2023-01-26', run_name)
results_files = os.listdir(results_path)

start_idx = 1550 if kitti_seq == '0028' else 0
gtpath = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'oxts', 'data')
gt_enu, gt_Rs, gt_attitudes = process_kitti_gt(gtpath, start_idx=start_idx)

data_path = os.path.join(os.getcwd(), '..', 'data', 'kitti', kitti_seq, 'icp')
lidar_Rs, lidar_ts, lidar_positions, lidar_covariances = load_icp_results(data_path, start_idx=start_idx)

N_SHIFT = 10

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['fgo_positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    spoofed_positions = results['spoofed_pos']

    qs = results['qs']
    threshold = results['threshold']
    if any(qs > threshold):
        detect_idx = N_SHIFT * np.argmax(qs > threshold)
    else:
        detect_idx = None

    plot_trajectories(gt_enu, graph_positions, lidar_positions[:graph_positions.shape[0]], spoofed_positions, detect_idx)

Monte carlo errors

In [ ]:
from plotly.subplots import make_subplots

def plot_errors(fig, gt_enu, graph_positions):
    traj_len = len(gt_enu)
    
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,0] - gt_enu[:,0], name='x error'), row=1, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,1] - gt_enu[:,1], name='y error'), row=2, col=1)
    fig.add_trace(go.Scatter(x=np.arange(traj_len), y=graph_positions[:,2] - gt_enu[:,2], name='z error'), row=3, col=1)

In [ ]:
# Error envelope
fig = make_subplots(rows=3, cols=1, shared_xaxes=True, vertical_spacing=0.05)

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))
    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]
    plot_errors(fig, gt_enu, graph_positions)

fig.update_layout(width=1200, height=700)
fig.update_layout(font=dict(size=15))
fig.show()

Error statistics

In [ ]:
np.set_printoptions(suppress=True, precision=3)

In [ ]:
# Nominal

false_alarms = 0
total_trials = 0
trajectory_false_alarms = 0

ss_start = 0  # Start of steady state
xyz_errors = []

for fname in results_files:
    results = np.load(os.path.join(results_path, fname))

    graph_positions = results['positions']
    gt_enu = gt_enu[:graph_positions.shape[0]]

    xyz_errors.append(gt_enu[ss_start:] - graph_positions[ss_start:])

    # False alarm
    qs = results['qs']
    threshold = results['threshold']
    false_alarms += np.sum(qs > threshold)
    total_trials += len(qs)
    if any(qs > threshold):
        trajectory_false_alarms += 1


print(f'Per trial false alarm rate: {false_alarms} of {total_trials}')
print(f'Per trajectory false alarm rate: {trajectory_false_alarms} of {len(results_files)}')

In [ ]:
xyz_errors = np.vstack(xyz_errors)

In [ ]:
print("mean: ", np.mean(xyz_errors, axis=0))
print("std: ", np.std(xyz_errors, axis=0))